In [1]:
import socket
import time


In [4]:
import socket,time,pickle
from data.Algorithem.global_repository import game_status_repository,player_status_repository,round_status_repository
class Nomal_Client():
    def __init__(self,name):
        self.name = name
        self.ip_list = []
        self.server_ip = "192.168.0.5"
        
    def get_list_Green_room(self):
        global game_status_repository,round_status_repository,player_status_repository
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip_address = s.getsockname()[0]
        s.close()
        myadress = ip_address
        self.ip_list = []
        for i in range(255):
            ip =  ".".join(myadress.split(".")[:-1])+f".{i}"
            # try:
            try:
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.settimeout(0.01)  # 타임아웃을 0.01초로 설정
                s.connect((ip, 12345))
                t = time.time()
                s.send(pickle.dumps(["ping",self.name]))
                response = pickle.loads(s.recv(1024))
                print((ip,response))
                s.close()
                self.ip_list.append((ip,response))
                print(f"connecting ip : {ip}")
            except:pass
            # except (socket.timeout, socket.error):
            #     pass
        return self.ip_list
    
    def send_client(self,message, server_port=12345):
        global game_status_repository,round_status_repository,player_status_repository
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client_socket.connect((self.server_ip, server_port))
        if message == "process":
            message =[game_status_repository,round_status_repository,player_status_repository,"",""]
        else:
            message = [message]
        message.append(self.name)
        message.append(self.name)
        t = time.time()
        self.client_socket.send(pickle.dumps(message))
        response = self.client_socket.recv(9192)
        print(len(response))
        self.client_socket.close()
        response = pickle.loads(response)
        print(response)

        if len(response) < 4:
            print(f"ping :{time.time() - t}ms")
            print(f'Received from server: {response}')
            #입장 완료
        else:
            game_status_repository = response[0]
            round_status_repository = response[1]
            player_status_repository = response[2]
            print(game_status_repository.now_turn_player, self.name)
            if game_status_repository.now_turn_player != self.name and\
               game_status_repository.status == "start":
                self.send_client("process")
            if game_status_repository.status == "greenroom" and player_status_repository[self.name].ready:
                self.send_client("process")
 

In [5]:
client = Nomal_Client("김성훈")
client.send_client("JOIN")


2424
[<data.Algorithem.repository.game_status.GameStatus object at 0x00000258819BF830>, <data.Algorithem.repository.round_status.RoundStatus object at 0x000002588314B230>, {'김성훈': <data.Algorithem.repository.player_status.PlayerStatus object at 0x000002588314A480>}, '', '']
0 김성훈


In [6]:
player_status_repository[client.name].ready = True
client.send_client("process")

3124
[<data.Algorithem.repository.game_status.GameStatus object at 0x0000025882FCF890>, <data.Algorithem.repository.round_status.RoundStatus object at 0x000002588314A390>, {'김성훈': <data.Algorithem.repository.player_status.PlayerStatus object at 0x0000025883149310>, '김도훈': <data.Algorithem.repository.player_status.PlayerStatus object at 0x0000025883148A70>}, '', '']
김성훈 김성훈


In [11]:
game_status_repository.next = True
game_status_repository.next_player()
print(game_status_repository.now_turn_player)
client.send_client("process")

김도훈
3144
[<data.Algorithem.repository.game_status.GameStatus object at 0x0000025881C5E900>, <data.Algorithem.repository.round_status.RoundStatus object at 0x000002588314BEC0>, {'김성훈': <data.Algorithem.repository.player_status.PlayerStatus object at 0x000002588314A510>, '김도훈': <data.Algorithem.repository.player_status.PlayerStatus object at 0x000002588314A1E0>}, '', '']
김도훈 김성훈
3144
[<data.Algorithem.repository.game_status.GameStatus object at 0x00000258819BF830>, <data.Algorithem.repository.round_status.RoundStatus object at 0x0000025883149F70>, {'김성훈': <data.Algorithem.repository.player_status.PlayerStatus object at 0x0000025883148830>, '김도훈': <data.Algorithem.repository.player_status.PlayerStatus object at 0x000002588314BF80>}, '', '']
김성훈 김성훈


3144
[<data.Algorithem.repository.game_status.GameStatus object at 0x0000025882FCF830>, <data.Algorithem.repository.round_status.RoundStatus object at 0x000002588314B620>, {'김성훈': <data.Algorithem.repository.player_status.PlayerStatus object at 0x000002588314A510>, '김도훈': <data.Algorithem.repository.player_status.PlayerStatus object at 0x000002588314B320>}, '', '']
김도훈 김성훈
3144
[<data.Algorithem.repository.game_status.GameStatus object at 0x000002588314A180>, <data.Algorithem.repository.round_status.RoundStatus object at 0x000002588314AF30>, {'김성훈': <data.Algorithem.repository.player_status.PlayerStatus object at 0x00000258831491F0>, '김도훈': <data.Algorithem.repository.player_status.PlayerStatus object at 0x000002588314B650>}, '', '']
김성훈 김성훈


In [20]:
player_status_repository[client.name] = True

TypeError: list indices must be integers or slices, not str

In [24]:
game_status_repository.now_turn_player = 1

In [25]:
client.send_client([game_status_repository,round_status_repository,player_status_repository])

4314
h
h
1 2
0
ping :8.381637334823608ms
Received from server: 


In [26]:
game_status_repository.now_turn_player

1

In [23]:
client.send_client([game_status_repository,round_status_repository,player_status_repository])

4314
h
h
2 2


In [15]:
game_status_repository.now_turn_player

1

In [ ]:
client.send_client([game_status_repository,round_status_repository,player_status_repository])

ConnectionResetError: [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다

In [ ]:
client.send_client("exit")

Received from server: yes


In [ ]:
from data.Algorithem import *
from data.Algorithem.global_repository import game_status_repository,round_status_repository,player_status_repository

In [ ]:
import pickle

In [ ]:
g = game_status_repository = data[0]
r = round_status_repository = data[1]
p=player_status_repository = data[2]

In [ ]:
p[0].card.hand_sub_card

['Pincer', 'Basket', 'Pitchfork']

In [ ]:
game_status_repository.round_card_order

[3, 1, 0, 2, 5, 6, 4, 8, 7, 9, 10, 11, 12, 13]

In [ ]:
g.now_turn_player

0

In [ ]:
p[0].card.hand_sub_card

['JunkWarehouse', 'Manger', 'Pincer']

In [ ]:
game_status_repository.round_card_order

[2, 1, 3, 0, 6, 5, 4, 8, 7, 9, 10, 11, 12, 13]